In [ ]:
!pip install hypernetx matplotlib scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.8/583.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.7 MB/s eta 0:00:00
  Attempting uninstall: decorator
    Found existing installation: decorator 4.4.2
    Uninstalling decorator-4.4.2:
      Successfully uninstalled decorator-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
moviepy 1.0.3 requires decorator<5.0,>=4.0.2, but you have decorator 5.1.1 which is incompatible.


In [ ]:
!pip install networkx numpy scikit-learn matplotlib

In [ ]:
!pip install python-louvain

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# ***Common Communities (Coverting to bipartite)***

In [ ]:
import networkx as nx
import hypernetx as hnx
import community as community_louvain  # For Louvain on graph
from collections import defaultdict

In [ ]:
# Step 1: Load undirected YouTube network data and create both graph and hypergraph representations
def load_youtube_network_data(filename):
    """Load the YouTube network data and return graph and hypergraph representations."""
    G = nx.Graph()  # For the graph-based representation
    hyperedges = defaultdict(list)  # For the hypergraph-based representation

    with open(filename, 'r') as file:
        for line in file:
            if not line.startswith('#'):  # Ignore comments
                nodes = list(map(int, line.strip().split()))
                u, v = nodes[0], nodes[1:]
                G.add_edges_from([(u, vi) for vi in v])  # Add edges to graph

                # For hypergraph, treat the first node as a "hyperedge" connected to all subsequent nodes
                hyperedges[u] = v

    H = hnx.Hypergraph(hyperedges)  # Create the hypergraph using HyperNetX
    return G, H

# Step 2: Convert hypergraph to bipartite graph manually
def convert_hypergraph_to_bipartite(H):
    """Convert hypergraph to a bipartite graph."""
    B = nx.Graph()  # Bipartite graph

    for edge_id, nodes in H.incidence_dict.items():
        # Add an edge between the hyperedge (edge_id) and each node in the hyperedge
        for node in nodes:
            B.add_edge(f"edge_{edge_id}", f"node_{node}")

    return B

In [ ]:
# Step 3: Apply Louvain community detection on graph
def apply_louvain_on_graph(G):
    """Apply Louvain algorithm for community detection on graph and return partitions."""
    partition = community_louvain.best_partition(G)
    return partition

# Step 4: Apply Louvain-like community detection on hypergraph (via bipartite graph)
def apply_louvain_on_hypergraph(H):
    """Apply Louvain-like algorithm on hypergraph (based on converting to bipartite graph) and return partitions."""
    bipartite_graph = convert_hypergraph_to_bipartite(H)  # Convert hypergraph to bipartite graph
    partition = community_louvain.best_partition(bipartite_graph)  # Apply Louvain on bipartite graph
    return partition


In [ ]:
# Step 5: Find common communities between graph and hypergraph partitions
def find_common_communities(graph_partition, hypergraph_partition):
    """Find common communities between the graph and hypergraph partitions."""
    common_communities = defaultdict(list)

    for node in graph_partition:
        # Ensure that the node exists in the hypergraph partition
        hyper_node = f"node_{node}"  # In bipartite graph, nodes are prefixed with "node_"
        if hyper_node in hypergraph_partition and graph_partition[node] == hypergraph_partition[hyper_node]:
            common_communities[graph_partition[node]].append(node)

    return common_communities

In [ ]:
def main():
    filename = '/content/drive/My Drive/CSE551/Undirected_Youtube_network.txt' # Path to the YouTube network data

    # Step 1: Load data and create graph/hypergraph
    G, H = load_youtube_network_data(filename)
    print(f"Loaded graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    print(f"Loaded hypergraph with {len(H.edges)} hyperedges.")

    # Step 2: Apply community detection on graph
    graph_partition = apply_louvain_on_graph(G)
    print(f"Detected {len(set(graph_partition.values()))} communities in the graph.")

    # Step 3: Apply community detection on hypergraph
    hypergraph_partition = apply_louvain_on_hypergraph(H)
    print(f"Detected {len(set(hypergraph_partition.values()))} communities in the hypergraph.")

    # Step 4: Find common communities
    common_communities = find_common_communities(graph_partition, hypergraph_partition)
    print(f"Found {len(common_communities)} common communities between graph and hypergraph.")

    # Print common communities
    for community, nodes in common_communities.items():
        print(f"Community {community} (common): Nodes = {nodes}")

if __name__ == "__main__":
    main()

Loaded graph with 1134890 nodes and 2987624 edges.
Loaded hypergraph with 374785 hyperedges.
Detected 5788 communities in the graph.
Detected 291987 communities in the hypergraph.
Found 2 common communities between graph and hypergraph.
Community 0 (common): Nodes = [665258]
Community 1838 (common): Nodes = [932537]




---



---



In [ ]:


# Step 1: Load undirected YouTube network data and create both graph and hypergraph representations
def load_youtube_network_data(filename):
    """Load the YouTube network data and return graph and hypergraph representations."""
    G = nx.Graph()  # For the graph-based representation
    hyperedges = defaultdict(list)  # For the hypergraph-based representation

    with open(filename, 'r') as file:
        for line in file:
            if not line.startswith('#'):  # Ignore comments
                nodes = list(map(int, line.strip().split()))
                u, v = nodes[0], nodes[1:]
                G.add_edges_from([(u, vi) for vi in v])  # Add edges to graph

                # For hypergraph, treat the first node as a "hyperedge" connected to all subsequent nodes
                hyperedges[u] = v

    H = hnx.Hypergraph(hyperedges)  # Create the hypergraph using HyperNetX
    return G, H

# Step 2: Convert hypergraph to bipartite graph manually
def convert_hypergraph_to_bipartite(H):
    """Convert hypergraph to a bipartite graph."""
    B = nx.Graph()  # Bipartite graph

    for edge_id, nodes in H.incidence_dict.items():
        # Add an edge between the hyperedge (edge_id) and each node in the hyperedge
        for node in nodes:
            B.add_edge(f"edge_{edge_id}", f"node_{node}")

    return B

# Step 3: Apply Louvain community detection on graph
def apply_louvain_on_graph(G):
    """Apply Louvain algorithm for community detection on graph and return partitions."""
    partition = community_louvain.best_partition(G)
    return partition

# Step 4: Apply Louvain-like community detection on hypergraph (via bipartite graph)
def apply_louvain_on_hypergraph(H):
    """Apply Louvain-like algorithm on hypergraph (based on converting to bipartite graph) and return partitions."""
    bipartite_graph = convert_hypergraph_to_bipartite(H)  # Convert hypergraph to bipartite graph
    partition = community_louvain.best_partition(bipartite_graph)  # Apply Louvain on bipartite graph
    return partition

# Step 5: Find common communities between graph and hypergraph partitions
def find_common_communities(graph_partition, hypergraph_partition):
    """Find common communities between the graph and hypergraph partitions."""
    common_communities = defaultdict(list)

    # Convert hypergraph partition node names back to their original form
    hypergraph_partition_cleaned = {
        int(node.split('_')[1]): community
        for node, community in hypergraph_partition.items() if node.startswith('node_')
    }

    for node in graph_partition:
        if node in hypergraph_partition_cleaned and graph_partition[node] == hypergraph_partition_cleaned[node]:
            common_communities[graph_partition[node]].append(node)

    return common_communities


In [ ]:
# Main function
def main():
    filename = '/content/drive/My Drive/CSE551/Undirected_Youtube_network.txt' # Path to the YouTube network data

    # Step 1: Load data and create graph/hypergraph
    G, H = load_youtube_network_data(filename)
    print(f"Loaded graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    print(f"Loaded hypergraph with {len(H.edges)} hyperedges.")

    # Step 2: Apply community detection on graph
    graph_partition = apply_louvain_on_graph(G)
    print(f"Detected {len(set(graph_partition.values()))} communities in the graph.")

    # Step 3: Apply community detection on hypergraph
    hypergraph_partition = apply_louvain_on_hypergraph(H)
    print(f"Detected {len(set(hypergraph_partition.values()))} communities in the hypergraph.")

    # Step 4: Find common communities
    common_communities = find_common_communities(graph_partition, hypergraph_partition)
    print(f"Found {len(common_communities)} common communities between graph and hypergraph.")

    # Step 5: Print common communities
    for community, nodes in common_communities.items():
        print(f"Community {community} (common): Nodes = {nodes}")

if __name__ == "__main__":
    main()


Loaded graph with 1134890 nodes and 2987624 edges.
Loaded hypergraph with 374785 hyperedges.
Detected 7191 communities in the graph.
Detected 291987 communities in the hypergraph.
Found 1 common communities between graph and hypergraph.
Community 0 (common): Nodes = [665258]


# ***Simillar communities***

In [ ]:
import networkx as nx
import hypernetx as hnx
import community as community_louvain  # For Louvain on graph
from collections import defaultdict
from itertools import combinations

In [ ]:
# Step 1: Load undirected YouTube network data and create both graph and hypergraph representations
def load_youtube_network_data(filename):
    """Load the YouTube network data and return graph and hypergraph representations."""
    G = nx.Graph()  # For the graph-based representation
    hyperedges = defaultdict(list)  # For the hypergraph-based representation

    with open(filename, 'r') as file:
        for line in file:
            if not line.startswith('#'):  # Ignore comments
                nodes = list(map(int, line.strip().split()))
                u, v = nodes[0], nodes[1:]
                G.add_edges_from([(u, vi) for vi in v])  # Add edges to graph

                # For hypergraph, treat the first node as a "hyperedge" connected to all subsequent nodes
                hyperedges[u] = v

    H = hnx.Hypergraph(hyperedges)  # Create the hypergraph using HyperNetX
    return G, H

# Step 2: Convert hypergraph to bipartite graph manually
def convert_hypergraph_to_bipartite(H):
    """Convert hypergraph to a bipartite graph."""
    B = nx.Graph()  # Bipartite graph

    for edge_id, nodes in H.incidence_dict.items():
        # Add an edge between the hyperedge (edge_id) and each node in the hyperedge
        for node in nodes:
            B.add_edge(f"edge_{edge_id}", f"node_{node}")

    return B

# Step 3: Apply Louvain community detection on graph
def apply_louvain_on_graph(G):
    """Apply Louvain algorithm for community detection on graph and return partitions."""
    partition = community_louvain.best_partition(G)
    return partition

# Step 4: Apply Louvain-like community detection on hypergraph (via bipartite graph)
def apply_louvain_on_hypergraph(H):
    """Apply Louvain-like algorithm on hypergraph (based on converting to bipartite graph) and return partitions."""
    bipartite_graph = convert_hypergraph_to_bipartite(H)  # Convert hypergraph to bipartite graph
    partition = community_louvain.best_partition(bipartite_graph)  # Apply Louvain on bipartite graph
    return partition

In [ ]:
# Helper function: Group nodes by community
def group_nodes_by_community(partition):
    """Group nodes by their community from the partition."""
    communities = defaultdict(list)
    for node, community in partition.items():
        communities[community].append(node)
    return communities

# Step 5: Calculate Jaccard similarity between two sets
def jaccard_similarity(set1, set2):
    """Calculate Jaccard similarity between two sets."""
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

In [ ]:
# Step 6: Find similar communities based on Jaccard similarity
def find_similar_communities(graph_partition, hypergraph_partition, threshold=0.1):
    """Find similar communities between the graph and hypergraph partitions."""
    graph_communities = group_nodes_by_community(graph_partition)
    hypergraph_communities = group_nodes_by_community(hypergraph_partition)

    similar_communities = []

    for g_comm, g_nodes in graph_communities.items():
        g_set = set(g_nodes)
        for h_comm, h_nodes in hypergraph_communities.items():
            h_set = set(int(node.split('_')[1]) for node in h_nodes if node.startswith('node_'))
            similarity = jaccard_similarity(g_set, h_set)
            if similarity > threshold:
                similar_communities.append((g_comm, h_comm, similarity, g_set & h_set))

    return similar_communities

# Step 7: Find the community pair with the maximum number of shared nodes
def find_maximum_shared_nodes(similar_communities):
    """Find the pair of communities with the maximum number of shared nodes."""
    max_shared_nodes = 0
    max_shared_pair = None

    for g_comm, h_comm, similarity, common_nodes in similar_communities:
        if len(common_nodes) > max_shared_nodes:
            max_shared_nodes = len(common_nodes)
            max_shared_pair = (g_comm, h_comm, common_nodes)

    return max_shared_pair, max_shared_nodes

In [ ]:
# Main function
def main():
    filename = '/content/drive/My Drive/CSE551/Undirected_Youtube_network.txt' # Path to the YouTube network data

    # Step 1: Load data and create graph/hypergraph
    G, H = load_youtube_network_data(filename)
    print(f"Loaded graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    print(f"Loaded hypergraph with {len(H.edges)} hyperedges.")

    # Step 2: Apply community detection on graph
    graph_partition = apply_louvain_on_graph(G)
    graph_communities = group_nodes_by_community(graph_partition)
    print(f"Detected {len(graph_communities)} communities in the graph.")

    # Step 3: Apply community detection on hypergraph
    hypergraph_partition = apply_louvain_on_hypergraph(H)
    hypergraph_communities = group_nodes_by_community(hypergraph_partition)
    print(f"Detected {len(hypergraph_communities)} communities in the hypergraph.")

    # Step 4: Print information about the communities in the graph
    print("\nGraph communities:")
    for comm, nodes in graph_communities.items():
        print(f"Community {comm}: {nodes}")

    # Step 5: Print information about the communities in the hypergraph
    print("\nHypergraph communities:")
    for comm, nodes in hypergraph_communities.items():
        print(f"Community {comm}: {nodes}")

    # Step 6: Find similar communities based on Jaccard similarity
    similar_communities = find_similar_communities(graph_partition, hypergraph_partition, threshold=0.2)
    print(f"\nFound {len(similar_communities)} similar communities between graph and hypergraph with a threshold of 0.2.")

    # Step 7: Print similar communities and shared nodes
    for g_comm, h_comm, similarity, common_nodes in similar_communities:
        print(f"Graph community {g_comm} and Hypergraph community {h_comm} have a Jaccard similarity of {similarity:.2f}.")
        print(f"Common nodes: {common_nodes}")

    # Step 8: Find and print the community pair with the maximum number of shared nodes
    max_shared_pair, max_shared_nodes = find_maximum_shared_nodes(similar_communities)
    if max_shared_pair:
        g_comm, h_comm, common_nodes = max_shared_pair
        print(f"\nCommunity pair with the maximum number of shared nodes:")
        print(f"Graph community {g_comm} and Hypergraph community {h_comm} share {max_shared_nodes} nodes.")
        print(f"Shared nodes: {common_nodes}")
    else:
        print("\nNo communities with shared nodes found.")

if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
Community 114373: ['edge_1152678', 'node_1152680']
Community 114374: ['edge_1152680', 'node_1152681']
Community 114376: ['edge_1152700', 'node_1157821']
Community 114378: ['edge_1152709', 'node_1152710']
Community 114380: ['edge_1152713', 'node_1152714']
Community 114382: ['edge_1152730', 'node_1157822']
Community 114383: ['edge_1152733', 'node_1152736']
Community 114386: ['edge_1152760', 'node_1152761']
Community 114388: ['edge_1152792', 'node_1152793']
Community 114390: ['edge_1152807', 'node_1152809']
Community 114391: ['edge_1152810', 'node_1152811']
Community 114393: ['edge_1152838', 'node_1157823']
Community 114395: ['edge_1152844', 'node_1152845']
Community 114397: ['edge_1152865', 'node_1157825']
Community 114399: ['edge_1152874', 'node_1152881']
Community 114402: ['edge_1152882', 'node_1157826']
Community 114403: ['edge_1152927', 'node_1152931', 'edge_1152930']
Community 114406: ['edge_1152936', 'node_1152949']
Community 11440

# ***Jacard Similarrities maximum number of shared nodes***

In [ ]:
import networkx as nx
import hypernetx as hnx
import community as community_louvain  # For Louvain on graph
from collections import defaultdict
from itertools import combinations

# Step 1: Load undirected YouTube network data and create both graph and hypergraph representations
def load_youtube_network_data(filename):
    """Load the YouTube network data and return graph and hypergraph representations."""
    G = nx.Graph()  # For the graph-based representation
    hyperedges = defaultdict(list)  # For the hypergraph-based representation

    with open(filename, 'r') as file:
        for line in file:
            if not line.startswith('#'):  # Ignore comments
                nodes = list(map(int, line.strip().split()))
                u, v = nodes[0], nodes[1:]
                G.add_edges_from([(u, vi) for vi in v])  # Add edges to graph

                # For hypergraph, treat the first node as a "hyperedge" connected to all subsequent nodes
                hyperedges[u] = v

    H = hnx.Hypergraph(hyperedges)  # Create the hypergraph using HyperNetX
    return G, H

# Step 2: Convert hypergraph to bipartite graph manually
def convert_hypergraph_to_bipartite(H):
    """Convert hypergraph to a bipartite graph."""
    B = nx.Graph()  # Bipartite graph

    for edge_id, nodes in H.incidence_dict.items():
        # Add an edge between the hyperedge (edge_id) and each node in the hyperedge
        for node in nodes:
            B.add_edge(f"edge_{edge_id}", f"node_{node}")

    return B

# Step 3: Apply Louvain community detection on graph
def apply_louvain_on_graph(G):
    """Apply Louvain algorithm for community detection on graph and return partitions."""
    partition = community_louvain.best_partition(G)
    return partition

# Step 4: Apply Louvain-like community detection on hypergraph (via bipartite graph)
def apply_louvain_on_hypergraph(H):
    """Apply Louvain-like algorithm on hypergraph (based on converting to bipartite graph) and return partitions."""
    bipartite_graph = convert_hypergraph_to_bipartite(H)  # Convert hypergraph to bipartite graph
    partition = community_louvain.best_partition(bipartite_graph)  # Apply Louvain on bipartite graph
    return partition

# Helper function: Group nodes by community
def group_nodes_by_community(partition):
    """Group nodes by their community from the partition."""
    communities = defaultdict(list)
    for node, community in partition.items():
        communities[community].append(node)
    return communities

# Step 5: Calculate Jaccard similarity between two sets
def jaccard_similarity(set1, set2):
    """Calculate Jaccard similarity between two sets."""
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

# Step 6: Find similar communities based on Jaccard similarity
def find_similar_communities(graph_partition, hypergraph_partition, threshold=0.1):
    """Find similar communities between the graph and hypergraph partitions."""
    graph_communities = group_nodes_by_community(graph_partition)
    hypergraph_communities = group_nodes_by_community(hypergraph_partition)

    similar_communities = []

    for g_comm, g_nodes in graph_communities.items():
        g_set = set(g_nodes)
        for h_comm, h_nodes in hypergraph_communities.items():
            h_set = set(int(node.split('_')[1]) for node in h_nodes if node.startswith('node_'))
            similarity = jaccard_similarity(g_set, h_set)
            if similarity > threshold:
                similar_communities.append((g_comm, h_comm, similarity, g_set & h_set))

    return similar_communities

# Step 7: Find the community pair with the maximum number of shared nodes
def find_maximum_shared_nodes(similar_communities):
    """Find the pair of communities with the maximum number of shared nodes."""
    max_shared_nodes = 0
    max_shared_pair = None

    for g_comm, h_comm, similarity, common_nodes in similar_communities:
        if len(common_nodes) > max_shared_nodes:
            max_shared_nodes = len(common_nodes)
            max_shared_pair = (g_comm, h_comm, common_nodes)

    return max_shared_pair, max_shared_nodes

# Main function
def main():
    filename = '/content/drive/My Drive/CSE551/Undirected_Youtube_network.txt' # Path to the YouTube network data

    # Step 1: Load data and create graph/hypergraph
    G, H = load_youtube_network_data(filename)
    print(f"Loaded graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
    print(f"Loaded hypergraph with {len(H.edges)} hyperedges.")

    # Step 2: Apply community detection on graph
    graph_partition = apply_louvain_on_graph(G)
    graph_communities = group_nodes_by_community(graph_partition)
    print(f"Detected {len(graph_communities)} communities in the graph.")

    # Step 3: Apply community detection on hypergraph
    hypergraph_partition = apply_louvain_on_hypergraph(H)
    hypergraph_communities = group_nodes_by_community(hypergraph_partition)
    print(f"Detected {len(hypergraph_communities)} communities in the hypergraph.")

    '''
    # Step 4: Print information about the communities in the graph
    print("\nGraph communities:")
    for comm, nodes in graph_communities.items():
        print(f"Community {comm}: {nodes}")

    # Step 5: Print information about the communities in the hypergraph
    print("\nHypergraph communities:")
    for comm, nodes in hypergraph_communities.items():
        print(f"Community {comm}: {nodes}")
    '''

    # Step 6: Find similar communities based on Jaccard similarity
    similar_communities = find_similar_communities(graph_partition, hypergraph_partition, threshold=0.2)
    print(f"\nFound {len(similar_communities)} similar communities between graph and hypergraph with a threshold of 0.2.")

    # Step 7: Print similar communities and shared nodes
    for g_comm, h_comm, similarity, common_nodes in similar_communities:
        print(f"Graph community {g_comm} and Hypergraph community {h_comm} have a Jaccard similarity of {similarity:.2f}.")
        print(f"Common nodes: {common_nodes}")

    # Step 8: Find and print the community pair with the maximum number of shared nodes
    max_shared_pair, max_shared_nodes = find_maximum_shared_nodes(similar_communities)
    if max_shared_pair:
        g_comm, h_comm, common_nodes = max_shared_pair
        print(f"\nCommunity pair with the maximum number of shared nodes:")
        print(f"Graph community {g_comm} and Hypergraph community {h_comm} share {max_shared_nodes} nodes.")
        print(f"Shared nodes: {common_nodes}")
    else:
        print("\nNo communities with shared nodes found.")

if __name__ == "__main__":
    main()


Streaming output truncated to the last 5000 lines.
Graph community 4339 and Hypergraph community 21483 have a Jaccard similarity of 0.33.
Common nodes: {956435}
Graph community 4339 and Hypergraph community 73002 have a Jaccard similarity of 0.33.
Common nodes: {1000878}
Graph community 4340 and Hypergraph community 92329 have a Jaccard similarity of 0.33.
Common nodes: {1131240}
Graph community 4340 and Hypergraph community 110581 have a Jaccard similarity of 0.33.
Common nodes: {1131242}
Graph community 4341 and Hypergraph community 161959 have a Jaccard similarity of 0.25.
Common nodes: {848386}
Graph community 4341 and Hypergraph community 46098 have a Jaccard similarity of 0.25.
Common nodes: {977883}
Graph community 4342 and Hypergraph community 161960 have a Jaccard similarity of 0.25.
Common nodes: {848393}
Graph community 4342 and Hypergraph community 250170 have a Jaccard similarity of 0.25.
Common nodes: {619880}
Graph community 4343 and Hypergraph community 161974 have a Ja



---



---



# **based on neighborhood structure**

In [ ]:
import networkx as nx
import hypernetx as hnx
import community as community_louvain  # For Louvain on graph
from collections import defaultdict

In [ ]:
# Load YouTube network data as graph and hypergraph
def load_youtube_network_data(filename):
    G = nx.Graph()
    hyperedges = defaultdict(list)

    with open(filename, 'r') as file:
        for line in file:
            if not line.startswith('#'):
                nodes = list(map(int, line.strip().split()))
                u, v = nodes[0], nodes[1:]
                G.add_edges_from([(u, vi) for vi in v])
                hyperedges[u] = v

    H = hnx.Hypergraph(hyperedges)
    return G, H

# Convert hypergraph to bipartite graph manually
def convert_hypergraph_to_bipartite(H):
    """Convert hypergraph to a bipartite graph."""
    B = nx.Graph()
    for edge_id, nodes in H.incidence_dict.items():
        for node in nodes:
            B.add_edge(f"edge_{edge_id}", f"node_{node}")
    return B

# Apply Louvain on graph
def apply_louvain_on_graph(G):
    partition = community_louvain.best_partition(G)
    return partition

# Apply Louvain on hypergraph via bipartite graph conversion
def apply_louvain_on_hypergraph(H):
    bipartite_graph = convert_hypergraph_to_bipartite(H)
    partition = community_louvain.best_partition(bipartite_graph)
    return partition


In [ ]:
# Group by neighborhood-based structure
def group_by_neighborhood(G, partition):
    community_groups = defaultdict(set)
    for node, community in partition.items():
        # Check if the node is a string prefixed with "node_"
        if isinstance(node, str) and node.startswith("node_"):
            actual_node = int(node.split('_')[1])  # Convert back to original node ID
        else:
            actual_node = node  # If integer, keep as-is

        # Get neighborhood and add to community groups
        neighborhood = set(G.neighbors(actual_node))
        neighborhood.add(actual_node)
        community_groups[community] |= neighborhood  # Union neighborhoods within a community

    return community_groups

# Calculate Jaccard similarity between two sets
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

In [ ]:
# Find similar communities based on neighborhood structure
def find_similar_communities(graph_communities, hypergraph_communities, threshold=0.2):
    similar_communities = []
    for g_comm, g_nodes in graph_communities.items():
        for h_comm, h_nodes in hypergraph_communities.items():
            similarity = jaccard_similarity(g_nodes, h_nodes)
            if similarity >= threshold:
                similar_communities.append((g_comm, h_comm, similarity, g_nodes & h_nodes))
    return similar_communities

In [ ]:
# Main function
def main():
    filename = '/content/drive/My Drive/CSE551/Undirected_Youtube_network.txt'

    # Load network as graph and hypergraph
    G, H = load_youtube_network_data(filename)

    # Apply community detection
    graph_partition = apply_louvain_on_graph(G)
    hypergraph_partition = apply_louvain_on_hypergraph(H)

    # Group by neighborhood-based communities
    graph_communities = group_by_neighborhood(G, graph_partition)
    hypergraph_communities = group_by_neighborhood(G, hypergraph_partition)

    # Find and print similar communities
    similar_communities = find_similar_communities(graph_communities, hypergraph_communities, threshold=0.2)
    print(f"\nSimilar communities based on neighborhood structure with Jaccard similarity >= 0.2:")
    for g_comm, h_comm, similarity, common_nodes in similar_communities:
        print(f"Graph community {g_comm} and Hypergraph community {h_comm} have similarity {similarity:.2f}.")
        print(f"Common nodes: {common_nodes}")

if __name__ == "__main__":
    main()


NetworkXError: The node edge_1 is not in the graph.